# **MODELO XGBOOST**

Utilizaremos uno de los algoritmos de boosting basados en árboles de decisión más utilizados en la actualidad para realizar las predicciones de la probabilidad de renovación.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [4]:
train = pd.read_csv("Dataset train.csv")
train.head()

,NUMPERIODO,nro_documento_hash,nro_telefono_hash,FECACTIVACIONCONTRATO_INT,FECINGRESOCLIENTE_INT,tipo1,tipo2,VCHMESADENDA,VCHPENALIDAD,alto,...,trafico_app_8,trafico_app_9,trafico_total,GIRO,SUBGIRO,TIENE_PROD_1,TIENE_PROD_2,TIENE_PROD_3,TARGET,PREV_TARGET
0,202201,5f58359a37ac456d9050c75aa36720b73063bf4b77f483...,9be57e3928a78ae85be270575f97e82a6bdbd59fc87281...,1072320.0,902100.0,1,0,0.0,0.00,1.0,...,0.000000,0.0,16606.140346,3e33e191fc2f1513037662f262867a04e278e8a0589a32...,8e0b81ed56b34766254d0492d32fa9d5d08a33690db2b6...,1,0,0,0.0,0.0
1,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,119dd6dff66e259a3996de7d885e2c54e8ed5874239346...,1072680.0,781560.0,1,0,7.6,365.57,1.0,...,180.536775,0.0,3705.222886,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
2,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,73cec44593f362cccdb45773197a0bf2136978cb241c91...,1072680.0,781560.0,1,0,0.0,0.00,0.0,...,980.521564,0.0,4617.043791,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
3,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,db86e518af696d60b247466fd1bb5823cf81a9442c16b2...,1072680.0,781560.0,0,1,0.0,0.00,0.0,...,0.000000,0.0,0.000000,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0
4,202201,ea411834dcd05b9e5eb4e1bed1a6b99250b33a4293fe46...,90d5ab1ad28da36b0fa821759b0a9df2b248dc92a557a5...,1072680.0,781560.0,1,0,0.0,0.00,1.0,...,37.027894,0.0,450.053533,a9d0bcb92394bd06ed0e020135054d2dbf3e512fdf0b7f...,e159ded5abcf126e18b05856319e4a4ee58a3267f298fd...,1,0,0,0.0,0.0


In [5]:
test = pd.read_csv("Dataset test.csv")
test.drop("TARGET", axis = 1, inplace = True)
test.head()

,NUMPERIODO,nro_documento_hash,nro_telefono_hash,FECACTIVACIONCONTRATO_INT,FECINGRESOCLIENTE_INT,tipo1,tipo2,VCHMESADENDA,VCHPENALIDAD,alto,...,trafico_app_7,trafico_app_8,trafico_app_9,trafico_total,GIRO,SUBGIRO,TIENE_PROD_1,TIENE_PROD_2,TIENE_PROD_3,PREV_TARGET
0,202204,5848c849c6e33ff61bc87f63781cec2cf5a38811446ec1...,ac8e7810e7d278b75f99a38183357eb28a2cb582f1875a...,1080060.0,1079280.0,1,0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,21631.486330,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0,0.0
1,202204,4623594982b1da7de047b2a10eba7533cc83a1b242167d...,93d2534f78d216261f0e0ce2e4d3e938f1d03fb8a4c766...,1080060.0,993180.0,1,0,0.0,0.0,1.0,...,0.00000,0.000000,0.0,5449.355104,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,0,0,0,0.0
2,202204,b38db111537bc0ebf7297373c3ba7114b6b74751195607...,0095dd8326c84eb25f2a2f888de98d29201dc9bd918829...,1080120.0,716760.0,0,1,0.0,0.0,1.0,...,0.00000,0.000000,0.0,7566.943466,404a18af35694f85854d87b70591b07c315b4ce5f5c7cf...,6eceace9a0758d06fca1f7c802c8ae57ae21361ae0e6a3...,1,0,0,0.0
3,202204,b996b613a4f6cc9b938241b75a4fbde47bb8606938159b...,255a24c2b9ecb57e21346616932f52c9a12924f7bf4738...,1080120.0,912240.0,1,0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,19392.310549,47a5a3efcbc0c9667600a36756e35afd8ed48df33f7e9d...,6b30ae99766ae521a61dccfbebed9aace3f62e0412aa6f...,1,0,0,0.0
4,202204,d1a5dd4012b5399691b0dfe3b24d4f6a303765e0364cc2...,28257d69713585f4741944029f476de65c9eda832e3063...,1079040.0,1007520.0,0,1,0.0,0.0,0.0,...,2.03855,218.083858,0.0,4611.995497,5b8e595b8cebb006e39a3d289e465a5718996785008cbb...,bd1b475ded03bdb3ab78de2f74782ad855c3c4b085d167...,1,0,0,0.0


Descartaremos las variables de tipo objeto para el entrenamiento.

In [6]:
cols_train = train.select_dtypes(exclude = 'object').columns
cols_train

Index(['NUMPERIODO', 'FECACTIVACIONCONTRATO_INT', 'FECINGRESOCLIENTE_INT',
       'tipo1', 'tipo2', 'VCHMESADENDA', 'VCHPENALIDAD', 'alto', 'bajo',
       'medio', 'muy alto', 'NRO_GRUPOS', 'grupo_2', 'grupo_1', 'grupo_11',
       'grupo_5', 'grupo_9', 'grupo_10', 'grupo_3', 'grupo_7', 'grupo_4',
       'grupo_8', 'grupo_6', 'MINUTOS1', 'GIGAS1', 'MENSAJES1', 'MINUTOS2',
       'GIGAS2', 'MENSAJES2', 'MINUTOS3', 'GIGAS3', 'MENSAJES3',
       'MINUTOS_TOTAL', 'MENSAJES_TOTAL', 'GIGAS_TOTAL', 'SMARTPHONE',
       'gamma_1', 'gamma_2', 'gamma_3', 'gamma_4', 'gamma_5', 'gamma_6',
       'gamma_7', 'LANZAMIENTO_INT', 'mins_flujo_1', 'mins_flujo_2',
       'trafico_app_1', 'trafico_app_2', 'trafico_app_3', 'trafico_app_4',
       'trafico_app_5', 'trafico_app_6', 'trafico_app_7', 'trafico_app_8',
       'trafico_app_9', 'trafico_total', 'TIENE_PROD_1', 'TIENE_PROD_2',
       'TIENE_PROD_3', 'TARGET', 'PREV_TARGET'],
      dtype='object')

Realizaremos la división de los datos del dataset de entrenamiento en 3 conjuntos:
* Train set: datos para entrenar los modelos (96%)
* Dev set: datos para comparar los modelos (2%: 11K)
* Test set: datos para probar los modelos (2%: 11K)

Los datasets de validación y prueba deben reflejar la proporción de registros positivos existentes en el dataset general para que el modelo pueda generalizar correctamente.

In [7]:
pos_records = train.loc[train['TARGET'] == 1, cols_train]
pos_records.shape

(16472, 61)

In [8]:
neg_records = train.loc[train['TARGET'] == 0, cols_train]
neg_records.shape

(531523, 61)

In [9]:
X_pos = pos_records.loc[:, cols_train != "TARGET"]
Y_pos = pos_records.loc[:, "TARGET"]
X_neg = neg_records.loc[:, cols_train != "TARGET"]
Y_neg = neg_records.loc[:, "TARGET"]

In [10]:
X_train_pos, X_rem_pos, Y_train_pos, Y_rem_pos = train_test_split(X_pos, Y_pos, test_size = 0.04, shuffle = True, random_state = 0)
X_dev_pos, X_test_pos, Y_dev_pos, Y_test_pos = train_test_split(X_rem_pos, Y_rem_pos, test_size = 0.5, shuffle = True, random_state = 0)

X_train_neg, X_rem_neg, Y_train_neg, Y_rem_neg = train_test_split(X_neg, Y_neg, test_size = 0.04, shuffle = True, random_state = 0)
X_dev_neg, X_test_neg, Y_dev_neg, Y_test_neg = train_test_split(X_rem_neg, Y_rem_neg, test_size = 0.5, shuffle = True, random_state = 0)

X_train = pd.concat([X_train_pos, X_train_neg], axis = 0)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis = 0)
X_dev = pd.concat([X_dev_pos, X_dev_neg], axis = 0)
Y_dev = pd.concat([Y_dev_pos, Y_dev_neg], axis = 0)
X_test = pd.concat([X_test_pos, X_test_neg], axis = 0)
Y_test = pd.concat([Y_test_pos, Y_test_neg], axis = 0)

In [11]:
print("X_train size: {}".format(X_train.shape))
print("Y_train size: {}".format(Y_train.shape))
print("X_dev size: {}".format(X_dev.shape))
print("Y_dev size: {}".format(Y_dev.shape))
print("X_test size: {}".format(X_test.shape))
print("Y_test size: {}".format(Y_test.shape))

X_train size: (526075, 60)
Y_train size: (526075,)
X_dev size: (10959, 60)
Y_dev size: (10959,)
X_test size: (10961, 60)
Y_test size: (10961,)


In [12]:
print("Proportion train set: {:.3f} %".format(Y_train.value_counts()[1]/Y_train.shape[0] * 100))
print("Proportion dev set: {:.3f} %".format(Y_dev.value_counts()[1]/Y_dev.shape[0] * 100))
print("Proportion test set: {:.3f} %".format(Y_test.value_counts()[1]/Y_test.shape[0] * 100))

Proportion train set: 3.006 %
Proportion dev set: 3.002 %
Proportion test set: 3.011 %


Iteraremos sobre 4 parámetros del modelo XGBoost:
* Max_depth: la profundidad máxima que pueden tener los árboles.
* Gamma: Reducción de pérdida mínima para crear una partición en un nodo hoja del árbol.
* Eta: Learning rate.
* Subsample: Fracción del dataset que será utilizada en cada árbol.
* Colsample: Fracción de las columnas del dataset que será utilizadas en cada árbol.

Utilizaremos un número pequeño de estimadores por cada modelo entrenado.

In [100]:
max_depth = np.arange(4, 9, 2).reshape(1, -1)
gammas = np.array([0.5, 1, 2, 5]).reshape(1, -1)
eta = np.logspace(-3, 0, 4).reshape(1, -1)
subsamples = np.array([0.6, 0.8, 1.0]).reshape(1, -1)
colsamples = np.array([0.6, 0.8, 1.0]).reshape(1, -1)

print("Máxima profundidad: {}".format(max_depth[0]))
print("Gamma: {}".format(gammas[0]))
print("Learning rate: {}".format(eta[0]))
print("Subsample: {}".format(subsamples[0]))
print("Colsample: {}".format(colsamples[0]))

Máxima profundidad: [4 6 8]
Gamma: [0.5 1.  2.  5. ]
Learning rate: [0.001 0.01  0.1   1.   ]
Subsample: [0.6 0.8 1. ]
Colsample: [0.6 0.8 1. ]


In [93]:
def combine_values(arr1, arr2):

    """ Realiza el producto cruz entre 2 arreglos

        Args:
            arr1 (np.array): arreglo 1, puede tener cualquier dimensión
            arr2 (np.array): arreglo 2, debe ser de la forma (1, m)

        Return:
            c (np.array): arreglo con el producto cruz de los 2 arreglos
    """

    a = np.repeat(arr2, arr1.shape[1]).reshape(1, -1)
    b = np.tile(arr1, arr2.shape[1]).reshape(-1, a.shape[1])

    c = np.concatenate((b, a))

    return c

In [101]:
parameters = combine_values(max_depth, gammas)
parameters = combine_values(parameters, eta)
parameters = combine_values(parameters, subsamples)
parameters = combine_values(parameters, colsamples).T
parameters.shape

(432, 5)

In [112]:
tree = xgb.XGBClassifier(booster = 'gbtree', n_estimators = 10, objective = 'binary:logistic', random_state = 0)
model = Pipeline([('scaler', MinMaxScaler()), ('tree', tree)])
results = []

for i in range(parameters.shape[0]):

    depth, gamma, lr, subsample, colsample = parameters[i]
    
    model.set_params(tree__max_depth = int(depth), tree__gamma = gamma, tree__learning_rate = lr,
                    tree__subsample = subsample, tree__colsample_bytree = colsample)

    model.fit(X_train, Y_train)

    y_train_preds = model.predict_proba(X_train)
    auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])
    
    y_dev_preds = model.predict_proba(X_dev)
    auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

    if i % 36 == 0:
        print("Depth: {}, gamma: {}, learning_rate: {}, subsample: {}, colsample: {}"
             .format(depth, gamma, lr, subsample, colsample))
        print("Train score: {}".format(auc_score_train))
        print("Dev score: {}".format(auc_score_dev))

    results.append([depth, gamma, lr, subsample, colsample, auc_score_train, auc_score_dev])


Depth: 4.0, gamma: 0.5, learning_rate: 0.001, subsample: 0.6, colsample: 0.6
Train score: 0.760120216597899
Dev score: 0.752622331132569
Depth: 4.0, gamma: 0.5, learning_rate: 1.0, subsample: 0.6, colsample: 0.6
Train score: 0.7772722029528364
Dev score: 0.7781432374394885
Depth: 4.0, gamma: 0.5, learning_rate: 0.1, subsample: 0.8, colsample: 0.6
Train score: 0.762985328614659
Dev score: 0.7504263325393807
Depth: 4.0, gamma: 0.5, learning_rate: 0.01, subsample: 1.0, colsample: 0.6
Train score: 0.7585826906951656
Dev score: 0.7507956205840556
Depth: 4.0, gamma: 0.5, learning_rate: 0.001, subsample: 0.6, colsample: 0.8
Train score: 0.7563928527871143
Dev score: 0.7569947416127436
Depth: 4.0, gamma: 0.5, learning_rate: 1.0, subsample: 0.6, colsample: 0.8
Train score: 0.7782882180884592
Dev score: 0.7677282566115856
Depth: 4.0, gamma: 0.5, learning_rate: 0.1, subsample: 0.8, colsample: 0.8
Train score: 0.7612066138721167
Dev score: 0.7535832234857474
Depth: 4.0, gamma: 0.5, learning_rate: 

In [113]:
cols = ['max depth', 'gamma', 'learning rate', 'subsample', 'colsample', 'Train score', 'Dev score']
results_df = pd.DataFrame(np.array(results), columns = cols)
results_df['Train score'] = results_df['Train score'].astype(np.float32)
results_df['Dev score'] = results_df['Dev score'].astype(np.float32)

results_df['Variance'] = results_df['Train score'] - results_df['Dev score']

results_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head(10)

,max depth,gamma,learning rate,subsample,colsample,Train score,Dev score,Variance
427,6.0,2.0,1.0,1.0,1.0,0.804395,0.792871,0.011524
424,6.0,1.0,1.0,1.0,1.0,0.802840,0.788868,0.013972
379,6.0,2.0,1.0,0.8,1.0,0.799103,0.788724,0.010379
95,8.0,5.0,1.0,0.8,0.6,0.815839,0.785688,0.030151
136,6.0,1.0,1.0,1.0,0.6,0.803600,0.785651,0.017949
334,6.0,5.0,1.0,0.6,1.0,0.789526,0.783557,0.005969
133,6.0,0.5,1.0,1.0,0.6,0.803492,0.782564,0.020928
85,6.0,0.5,1.0,0.8,0.6,0.799367,0.782399,0.016967
373,6.0,0.5,1.0,0.8,1.0,0.801912,0.781474,0.020437
421,6.0,0.5,1.0,1.0,1.0,0.806173,0.781396,0.024777


Del top 10 de los modelos obtenidos podemos resaltar lo siguiente:

* La profundidad máxima óptima está entre 6 y 8.
* El valor del learning rate óptimo es 1.
* Los valores de subsample 0.8 y 1 son los que devuelven los mejores resultados, por lo que se utilizarán para la siguiente iteración.
* Los rangos de gamma y colsample serán los mismos en la siguiente iteración.

En la nueva iteración se utilizará un número mayor de árboles por modelo.

In [117]:
max_depth = np.arange(6, 9).reshape(1, -1)
gammas = np.array([0.5, 1, 2, 5]).reshape(1, -1)
subsamples = np.array([0.8, 1.0]).reshape(1, -1)
colsamples = np.array([0.6, 0.8, 1.0]).reshape(1, -1)

print("Máxima profundidad: {}".format(max_depth[0]))
print("Gamma: {}".format(gammas[0]))
print("Subsample: {}".format(subsamples[0]))
print("Colsample: {}".format(colsamples[0]))

Máxima profundidad: [6 7 8]
Gamma: [0.5 1.  2.  5. ]
Subsample: [0.8 1. ]
Colsample: [0.6 0.8 1. ]


In [118]:
parameters = combine_values(max_depth, gammas)
parameters = combine_values(parameters, subsamples)
parameters = combine_values(parameters, colsamples).T
parameters.shape

(72, 4)

In [123]:
tree2 = xgb.XGBClassifier(booster = 'gbtree', n_estimators = 50, learning_rate = 1, objective = 'binary:logistic', random_state = 0)
model2 = Pipeline([('scaler', MinMaxScaler()), ('tree', tree2)])
results2 = []

for i in range(parameters.shape[0]):

    depth, gamma, subsample, colsample = parameters[i]
    
    model2.set_params(tree__max_depth = int(depth), tree__gamma = gamma, tree__subsample = subsample, 
                     tree__colsample_bytree = colsample)

    model2.fit(X_train, Y_train)

    y_train_preds = model2.predict_proba(X_train)
    auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])
    
    y_dev_preds = model2.predict_proba(X_dev)
    auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

    if i % 9 == 0:
        print("Depth: {}, gamma: {}, subsample: {}, colsample: {}"
             .format(depth, gamma, subsample, colsample))
        print("Train score: {}".format(auc_score_train))
        print("Dev score: {}".format(auc_score_dev))

    results2.append([depth, gamma, subsample, colsample, auc_score_train, auc_score_dev])


Depth: 6.0, gamma: 0.5, subsample: 0.8, colsample: 0.6
Train score: 0.8470389371987248
Dev score: 0.7712308457739895
Depth: 6.0, gamma: 5.0, subsample: 0.8, colsample: 0.6
Train score: 0.8417157464895475
Dev score: 0.7559277665150246
Depth: 6.0, gamma: 2.0, subsample: 1.0, colsample: 0.6
Train score: 0.8587918038897921
Dev score: 0.7620729883594919
Depth: 6.0, gamma: 1.0, subsample: 0.8, colsample: 0.8
Train score: 0.8478165166392834
Dev score: 0.7523732797296176
Depth: 6.0, gamma: 0.5, subsample: 1.0, colsample: 0.8
Train score: 0.8634936894146157
Dev score: 0.7709213186285302
Depth: 6.0, gamma: 5.0, subsample: 1.0, colsample: 0.8
Train score: 0.8485209675509366
Dev score: 0.7714234245568687
Depth: 6.0, gamma: 2.0, subsample: 0.8, colsample: 1.0
Train score: 0.8509763306507868
Dev score: 0.7746916880881374
Depth: 6.0, gamma: 1.0, subsample: 1.0, colsample: 1.0
Train score: 0.8630915096782931
Dev score: 0.7597693343665202


In [125]:
results2_df = pd.DataFrame(np.array(results2), columns = ['max depth', 'gamma', 'subsample', 'colsample', 'Train score', 'Dev score'])
results2_df['Train score'] = results2_df['Train score'].astype(np.float32)
results2_df['Dev score'] = results2_df['Dev score'].astype(np.float32)

results2_df['Variance'] = results2_df['Train score'] - results2_df['Dev score']

results2_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head()

,max depth,gamma,subsample,colsample,Train score,Dev score,Variance
66,6.0,2.0,1.0,1.0,0.857421,0.783987,0.073435
64,7.0,1.0,1.0,1.0,0.892757,0.775613,0.117144
54,6.0,2.0,0.8,1.0,0.850976,0.774692,0.076285
43,7.0,2.0,1.0,0.8,0.886298,0.771963,0.114335
45,6.0,5.0,1.0,0.8,0.848521,0.771423,0.077098


Calculamos el score del conjunto de entrenamiento para los 5 mejores modelos.

In [126]:
top5 = results2_df.sort_values(by = ['Dev score', 'Train score'], ascending = False).head()
top5 = top5.reset_index()
top5.drop('index', axis = 1, inplace = True)

top5['Test score'] = np.zeros(top5.shape[0])

tree2 = xgb.XGBClassifier(booster = 'gbtree', n_estimators = 100, learning_rate = 1, objective = 'binary:logistic', random_state = 0)
model2 = Pipeline([('scaler', MinMaxScaler()), ('tree', tree2)])

for i in range(top5.shape[0]):

    depth = top5.loc[i, 'max depth']
    gamma = top5.loc[i, 'gamma']
    subsample = top5.loc[i, 'subsample']
    colsample = top5.loc[i, 'colsample']

    model2.set_params(tree__max_depth = int(depth), tree__gamma = gamma, tree__subsample = subsample, tree__colsample_bytree = colsample)

    model2.fit(X_train, Y_train)

    y_preds = model2.predict_proba(X_test)
    test_score = roc_auc_score(Y_test, y_preds[:,1])

    top5.loc[i, 'Test score'] = test_score

    print("Model {} score: {}".format(i, test_score))


Model 0 score: 0.7592714844807781
Model 1 score: 0.7427933744366817
Model 2 score: 0.7363868674516779
Model 3 score: 0.751351821288798
Model 4 score: 0.7528397510995573


In [127]:
top5.dtypes

max depth      float64
gamma          float64
subsample      float64
colsample      float64
Train score    float32
Dev score      float32
Variance       float32
Test score     float64
dtype: object

In [128]:
top5['Dev - Test'] = top5['Dev score'] - top5['Test score']

top5.sort_values(by = ['Test score'], ascending = False)

,max depth,gamma,subsample,colsample,Train score,Dev score,Variance,Test score,Dev - Test
0,6.0,2.0,1.0,1.0,0.857421,0.783987,0.073435,0.759271,0.024715
4,6.0,5.0,1.0,0.8,0.848521,0.771423,0.077098,0.752840,0.018584
3,7.0,2.0,1.0,0.8,0.886298,0.771963,0.114335,0.751352,0.020611
1,7.0,1.0,1.0,1.0,0.892757,0.775613,0.117144,0.742793,0.032820
2,6.0,2.0,0.8,1.0,0.850976,0.774692,0.076285,0.736387,0.038305


Nos quedaremos con los parámetros del modelo 0, ya que presenta el mayor valor del score del conjunto de prueba y la diferencia entre los scores de los distintos conjuntos es pequeña.

A medida que se incrementa el número de árboles por modelo, el train score aumenta, mientras que el dev score disminuye.

In [133]:
xgbtree = xgb.XGBClassifier(booster = 'gbtree', max_depth = 6, gamma = 2.0, subsample = 1.0, colsample_bytree = 1.0,
                         n_estimators = 100, learning_rate = 1, objective = 'binary:logistic', random_state = 0)
final_model = Pipeline([('scaler', MinMaxScaler()), ('tree', xgbtree)])

final_model.fit(X_train, Y_train)

y_train_preds = final_model.predict_proba(X_train)
auc_score_train = roc_auc_score(Y_train, y_train_preds[:,1])

y_dev_preds = final_model.predict_proba(X_dev)
auc_score_dev = roc_auc_score(Y_dev, y_dev_preds[:,1])

y_test_preds = final_model.predict_proba(X_test)
auc_score_test = roc_auc_score(Y_test, y_test_preds[:,1])

print("Train AUC score: {}".format(auc_score_train))
print("Dev AUC score: {}".format(auc_score_dev))
print("Test AUC score: {}".format(auc_score_test))

Train AUC score: 0.8946845233633285
Dev AUC score: 0.767469340371204
Test AUC score: 0.7592714844807781


Procedemos a guardar el modelo, para ello usaremos la librería *joblib*.

In [134]:
import joblib

joblib.dump(final_model, 'xgboost_model.pkl')

['xgboost_model.pkl']

Finalmente, realizaremos las predicciones para el mes de abril.

In [135]:
cols_test = test.select_dtypes(exclude = 'object').columns
cols_test

Index(['NUMPERIODO', 'FECACTIVACIONCONTRATO_INT', 'FECINGRESOCLIENTE_INT',
       'tipo1', 'tipo2', 'VCHMESADENDA', 'VCHPENALIDAD', 'alto', 'bajo',
       'medio', 'muy alto', 'NRO_GRUPOS', 'grupo_2', 'grupo_1', 'grupo_11',
       'grupo_5', 'grupo_9', 'grupo_10', 'grupo_3', 'grupo_7', 'grupo_4',
       'grupo_8', 'grupo_6', 'MINUTOS1', 'GIGAS1', 'MENSAJES1', 'MINUTOS2',
       'GIGAS2', 'MENSAJES2', 'MINUTOS3', 'GIGAS3', 'MENSAJES3',
       'MINUTOS_TOTAL', 'MENSAJES_TOTAL', 'GIGAS_TOTAL', 'SMARTPHONE',
       'gamma_1', 'gamma_2', 'gamma_3', 'gamma_4', 'gamma_5', 'gamma_6',
       'gamma_7', 'LANZAMIENTO_INT', 'mins_flujo_1', 'mins_flujo_2',
       'trafico_app_1', 'trafico_app_2', 'trafico_app_3', 'trafico_app_4',
       'trafico_app_5', 'trafico_app_6', 'trafico_app_7', 'trafico_app_8',
       'trafico_app_9', 'trafico_total', 'TIENE_PROD_1', 'TIENE_PROD_2',
       'TIENE_PROD_3', 'PREV_TARGET'],
      dtype='object')

In [136]:
target_abril = final_model.predict_proba(test.loc[:, cols_test])
target_abril

array([[0.96613073, 0.03386924],
       [0.97820836, 0.02179161],
       [0.9983231 , 0.00167692],
       ...,
       [0.9721923 , 0.02780773],
       [0.9798824 , 0.02011756],
       [0.96416223, 0.0358378 ]], dtype=float32)

In [137]:
import warnings
warnings.filterwarnings("ignore")

In [138]:
predictions = test[['nro_telefono_hash']]
predictions['TARGET'] = target_abril[:, 1]
predictions.head()

,nro_telefono_hash,TARGET
0,ac8e7810e7d278b75f99a38183357eb28a2cb582f1875a...,0.033869
1,93d2534f78d216261f0e0ce2e4d3e938f1d03fb8a4c766...,0.021792
2,0095dd8326c84eb25f2a2f888de98d29201dc9bd918829...,0.001677
3,255a24c2b9ecb57e21346616932f52c9a12924f7bf4738...,0.038059
4,28257d69713585f4741944029f476de65c9eda832e3063...,0.014357


In [139]:
predictions.to_csv('xgboost_predictions.csv', index = False)